# 需用到的知识点

    1、Wavelet transform 小波变换  https://www.zhihu.com/question/22864189/answer/40772083
        Consequently, wavelet is useful in handling highly irregular financial time series
            https://www.researchgate.net/publication/223249583_Forecasting_stock_markets_using_wavelet_transforms_and_recurrent_neural_networks_An_integrated_system_based_on_artificial_bee_colony_algorithm
            
        https://pywavelets.readthedocs.io/en/latest/regression/index.html
        
        https://blog.csdn.net/zhaoyuxia517/article/details/78005713 基于小波变换的时间序列预测，Python实现
        
        http://www.docin.com/p-1546109075.html
        
        http://www.docin.com/p-1072414208.html?docfrom=rrela
        
        https://www.zhihu.com/question/19725983/answer/13856998
        
        http://www.360doc.com/content/13/0614/15/10724725_292827411.shtml
            
    2、Stacked autoencoders
        https://blog.csdn.net/jningwei/article/details/78836823
    
    另注意： 
    什么Autoencoder啦、RBM啦，现在都已经 没人用了 。
    现在所常说的 pre-training (预训练) ，其实 专指 migration learning (迁移学习)，那是一种无比强大又省事儿的trick。
    
        https://blog.csdn.net/hai008007/article/details/79676994

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 准备数据

In [1]:
# 包含部分指标数据
from stock.common.sdata import StockData
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
code = '600016'
sd = StockData(code)
data = sd.combine_income(ndays=5)
key_defines = ['RSI','MACD']
# 指标数据

data = StockData.combine_index_data(data,index_keys=key_defines)

use_column = [x not in ['Flag','INCOME'] for x in data.columns.get_level_values(0)]

data.iloc[:,use_column] =  \
     data.iloc[:,use_column].apply(lambda x: (x-np.mean(x))/np.std(x))

data_values = data.iloc[:,use_column].values
dlen = int(len(data_values)*0.8)
train_data = torch.from_numpy(np.array(data_values[0:dlen])).float()
test_data = torch.from_numpy(np.array(data_values[dlen:])).float()
dataset = TensorDataset(train_data)
train_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=True)
# data_in , target = sd.package_data(data=data, pdays=50)
# data_in = [x.values for x in data_in]
# dlen = int(len(data_in) * 0.9)
# train_data, train_target = torch.from_numpy(np.array(data_in[0:dlen])).float(), torch.from_numpy(np.array(target[0:dlen])).long()
# test_data, test_target = torch.from_numpy(np.array(data_in[dlen:])).float(), torch.from_numpy(np.array(target[dlen:]).squeeze()).long()

In [19]:
print(len(test_data))

565


# Autoencode模型

In [2]:
# 自编码主要用于数据降维
import torch.nn as nn
class AutoEncoder(nn.Module):
    def __init__(self, n_feature):
        super(AutoEncoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_feature, 6),
            nn.Tanh(),
            nn.Linear(6, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3,6),
            nn.Tanh(),
            nn.Linear(6, n_feature)
        )
    def forward(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded        
    
   

# 模型训练

In [ ]:
import torch
import os
import copy
LR = 0.00001
autoencoder = AutoEncoder(n_feature = 10)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR)
loss_func = nn.MSELoss()
epcho = 50000

if os.path.exists('/home/hecong/dplearn/model/autoencode_model.pkl'):
    print('加载上次训练模型...')
    autoencoder.load_state_dict(torch.load('/home/hecong/dplearn/model/autoencode_model.pkl'))
    
for epcho in range(epcho):
    run_loss = 0
    for step, x in enumerate(train_loader):
#         print(x[0])
#         print(x[0].shape)
        in_x = x[0]
        target  = x[0]
        encode,decode = autoencoder(in_x)
        loss = loss_func(decode, target)
        run_loss = run_loss + loss
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients        
        
    if epcho % 5 == 0:
        print('epcho --> {} ： loss {:.3f}'.format(epcho, run_loss/step))
        _wts = copy.deepcopy(autoencoder.state_dict())  
        torch.save(_wts,'/home/hecong/dplearn/model/autoencode_model.pkl')            
        

加载上次训练模型...
epcho --> 0 ： loss 0.056
epcho --> 5 ： loss 0.056
epcho --> 10 ： loss 0.056
epcho --> 15 ： loss 0.056
epcho --> 20 ： loss 0.056
epcho --> 25 ： loss 0.056
epcho --> 30 ： loss 0.056
epcho --> 35 ： loss 0.056
epcho --> 40 ： loss 0.056
epcho --> 45 ： loss 0.056
epcho --> 50 ： loss 0.056
epcho --> 55 ： loss 0.056
epcho --> 60 ： loss 0.056
epcho --> 65 ： loss 0.056
epcho --> 70 ： loss 0.056
epcho --> 75 ： loss 0.056
epcho --> 80 ： loss 0.056
epcho --> 85 ： loss 0.056
epcho --> 90 ： loss 0.056
epcho --> 95 ： loss 0.056
epcho --> 100 ： loss 0.056
epcho --> 105 ： loss 0.056
epcho --> 110 ： loss 0.056
epcho --> 115 ： loss 0.056
epcho --> 120 ： loss 0.056
epcho --> 125 ： loss 0.056
epcho --> 130 ： loss 0.056
epcho --> 135 ： loss 0.056
epcho --> 140 ： loss 0.056
epcho --> 145 ： loss 0.056
epcho --> 150 ： loss 0.056
epcho --> 155 ： loss 0.056
epcho --> 160 ： loss 0.056
epcho --> 165 ： loss 0.056
epcho --> 170 ： loss 0.056
epcho --> 175 ： loss 0.056
epcho --> 180 ： loss 0.056
epcho --> 1

epcho --> 1505 ： loss 0.056
epcho --> 1510 ： loss 0.056
epcho --> 1515 ： loss 0.056
epcho --> 1520 ： loss 0.056
epcho --> 1525 ： loss 0.056
epcho --> 1530 ： loss 0.056
epcho --> 1535 ： loss 0.056
epcho --> 1540 ： loss 0.056
epcho --> 1545 ： loss 0.056
epcho --> 1550 ： loss 0.057
epcho --> 1555 ： loss 0.056
epcho --> 1560 ： loss 0.056
epcho --> 1565 ： loss 0.056
epcho --> 1570 ： loss 0.056
epcho --> 1575 ： loss 0.056
epcho --> 1580 ： loss 0.056
epcho --> 1585 ： loss 0.056
epcho --> 1590 ： loss 0.056
epcho --> 1595 ： loss 0.056
epcho --> 1600 ： loss 0.056
epcho --> 1605 ： loss 0.056
epcho --> 1610 ： loss 0.056
epcho --> 1615 ： loss 0.056
epcho --> 1620 ： loss 0.056
epcho --> 1625 ： loss 0.056
epcho --> 1630 ： loss 0.056
epcho --> 1635 ： loss 0.056
epcho --> 1640 ： loss 0.056
epcho --> 1645 ： loss 0.056
epcho --> 1650 ： loss 0.056
epcho --> 1655 ： loss 0.056
epcho --> 1660 ： loss 0.056
epcho --> 1665 ： loss 0.056
epcho --> 1670 ： loss 0.056
epcho --> 1675 ： loss 0.056
epcho --> 1680 ： los

epcho --> 2970 ： loss 0.056
epcho --> 2975 ： loss 0.056
epcho --> 2980 ： loss 0.056
epcho --> 2985 ： loss 0.056
epcho --> 2990 ： loss 0.056
epcho --> 2995 ： loss 0.056
epcho --> 3000 ： loss 0.056
epcho --> 3005 ： loss 0.056
epcho --> 3010 ： loss 0.056
epcho --> 3015 ： loss 0.056
epcho --> 3020 ： loss 0.056
epcho --> 3025 ： loss 0.056
epcho --> 3030 ： loss 0.056
epcho --> 3035 ： loss 0.056
epcho --> 3040 ： loss 0.056
epcho --> 3045 ： loss 0.056
epcho --> 3050 ： loss 0.056
epcho --> 3055 ： loss 0.056
epcho --> 3060 ： loss 0.056
epcho --> 3065 ： loss 0.056
epcho --> 3070 ： loss 0.056
epcho --> 3075 ： loss 0.056
epcho --> 3080 ： loss 0.057
epcho --> 3085 ： loss 0.056
epcho --> 3090 ： loss 0.056
epcho --> 3095 ： loss 0.056
epcho --> 3100 ： loss 0.056
epcho --> 3105 ： loss 0.056
epcho --> 3110 ： loss 0.057
epcho --> 3115 ： loss 0.056
epcho --> 3120 ： loss 0.056
epcho --> 3125 ： loss 0.056
epcho --> 3130 ： loss 0.056
epcho --> 3135 ： loss 0.056
epcho --> 3140 ： loss 0.057
epcho --> 3145 ： los

epcho --> 4435 ： loss 0.056
epcho --> 4440 ： loss 0.056
epcho --> 4445 ： loss 0.057
epcho --> 4450 ： loss 0.056
epcho --> 4455 ： loss 0.056
epcho --> 4460 ： loss 0.056
epcho --> 4465 ： loss 0.056
epcho --> 4470 ： loss 0.056
epcho --> 4475 ： loss 0.057
epcho --> 4480 ： loss 0.056
epcho --> 4485 ： loss 0.056
epcho --> 4490 ： loss 0.056
epcho --> 4495 ： loss 0.056
epcho --> 4500 ： loss 0.056
epcho --> 4505 ： loss 0.056
epcho --> 4510 ： loss 0.056
epcho --> 4515 ： loss 0.056
epcho --> 4520 ： loss 0.056
epcho --> 4525 ： loss 0.057
epcho --> 4530 ： loss 0.057
epcho --> 4535 ： loss 0.056
epcho --> 4540 ： loss 0.056
epcho --> 4545 ： loss 0.056
epcho --> 4550 ： loss 0.056
epcho --> 4555 ： loss 0.056
epcho --> 4560 ： loss 0.056
epcho --> 4565 ： loss 0.056
epcho --> 4570 ： loss 0.056
epcho --> 4575 ： loss 0.056
epcho --> 4580 ： loss 0.056
epcho --> 4585 ： loss 0.056
epcho --> 4590 ： loss 0.056
epcho --> 4595 ： loss 0.056
epcho --> 4600 ： loss 0.056
epcho --> 4605 ： loss 0.056
epcho --> 4610 ： los

epcho --> 5900 ： loss 0.056
epcho --> 5905 ： loss 0.056
epcho --> 5910 ： loss 0.056
epcho --> 5915 ： loss 0.056
epcho --> 5920 ： loss 0.056
epcho --> 5925 ： loss 0.056
epcho --> 5930 ： loss 0.056
epcho --> 5935 ： loss 0.056
epcho --> 5940 ： loss 0.056
epcho --> 5945 ： loss 0.056
epcho --> 5950 ： loss 0.056
epcho --> 5955 ： loss 0.056
epcho --> 5960 ： loss 0.056
epcho --> 5965 ： loss 0.056
epcho --> 5970 ： loss 0.056
epcho --> 5975 ： loss 0.056
epcho --> 5980 ： loss 0.056
epcho --> 5985 ： loss 0.056
epcho --> 5990 ： loss 0.056
epcho --> 5995 ： loss 0.056
epcho --> 6000 ： loss 0.056
epcho --> 6005 ： loss 0.056
epcho --> 6010 ： loss 0.056
epcho --> 6015 ： loss 0.056
epcho --> 6020 ： loss 0.056
epcho --> 6025 ： loss 0.056
epcho --> 6030 ： loss 0.056
epcho --> 6035 ： loss 0.056
epcho --> 6040 ： loss 0.056
epcho --> 6045 ： loss 0.056
epcho --> 6050 ： loss 0.056
epcho --> 6055 ： loss 0.057
epcho --> 6060 ： loss 0.056
epcho --> 6065 ： loss 0.057
epcho --> 6070 ： loss 0.056
epcho --> 6075 ： los

epcho --> 7365 ： loss 0.056
epcho --> 7370 ： loss 0.056
epcho --> 7375 ： loss 0.056
epcho --> 7380 ： loss 0.056
epcho --> 7385 ： loss 0.056
epcho --> 7390 ： loss 0.056
epcho --> 7395 ： loss 0.056
epcho --> 7400 ： loss 0.056
epcho --> 7405 ： loss 0.056
epcho --> 7410 ： loss 0.056
epcho --> 7415 ： loss 0.056
epcho --> 7420 ： loss 0.056
epcho --> 7425 ： loss 0.056
epcho --> 7430 ： loss 0.056
epcho --> 7435 ： loss 0.056
epcho --> 7440 ： loss 0.056
epcho --> 7445 ： loss 0.056
epcho --> 7450 ： loss 0.056
epcho --> 7455 ： loss 0.056
epcho --> 7460 ： loss 0.056
epcho --> 7465 ： loss 0.056
epcho --> 7470 ： loss 0.056
epcho --> 7475 ： loss 0.056
epcho --> 7480 ： loss 0.056
epcho --> 7485 ： loss 0.057
epcho --> 7490 ： loss 0.056
epcho --> 7495 ： loss 0.056
epcho --> 7500 ： loss 0.056
epcho --> 7505 ： loss 0.056
epcho --> 7510 ： loss 0.056
epcho --> 7515 ： loss 0.056
epcho --> 7520 ： loss 0.056
epcho --> 7525 ： loss 0.056
epcho --> 7530 ： loss 0.056
epcho --> 7535 ： loss 0.056
epcho --> 7540 ： los

epcho --> 8830 ： loss 0.056
epcho --> 8835 ： loss 0.056
epcho --> 8840 ： loss 0.056
epcho --> 8845 ： loss 0.056
epcho --> 8850 ： loss 0.056
epcho --> 8855 ： loss 0.056
epcho --> 8860 ： loss 0.056
epcho --> 8865 ： loss 0.056
epcho --> 8870 ： loss 0.056
epcho --> 8875 ： loss 0.056
epcho --> 8880 ： loss 0.056
epcho --> 8885 ： loss 0.056
epcho --> 8890 ： loss 0.056
epcho --> 8895 ： loss 0.056
epcho --> 8900 ： loss 0.056
epcho --> 8905 ： loss 0.056
epcho --> 8910 ： loss 0.056
epcho --> 8915 ： loss 0.056
epcho --> 8920 ： loss 0.056
epcho --> 8925 ： loss 0.056
epcho --> 8930 ： loss 0.056
epcho --> 8935 ： loss 0.056
epcho --> 8940 ： loss 0.056
epcho --> 8945 ： loss 0.056
epcho --> 8950 ： loss 0.056
epcho --> 8955 ： loss 0.056
epcho --> 8960 ： loss 0.056
epcho --> 8965 ： loss 0.056
epcho --> 8970 ： loss 0.056
epcho --> 8975 ： loss 0.056
epcho --> 8980 ： loss 0.056
epcho --> 8985 ： loss 0.056
epcho --> 8990 ： loss 0.056
epcho --> 8995 ： loss 0.056
epcho --> 9000 ： loss 0.056
epcho --> 9005 ： los

epcho --> 10285 ： loss 0.056
epcho --> 10290 ： loss 0.056
epcho --> 10295 ： loss 0.056
epcho --> 10300 ： loss 0.056
epcho --> 10305 ： loss 0.056
epcho --> 10310 ： loss 0.056
epcho --> 10315 ： loss 0.056
epcho --> 10320 ： loss 0.056
epcho --> 10325 ： loss 0.056
epcho --> 10330 ： loss 0.056
epcho --> 10335 ： loss 0.056
epcho --> 10340 ： loss 0.056
epcho --> 10345 ： loss 0.056
epcho --> 10350 ： loss 0.056
epcho --> 10355 ： loss 0.056
epcho --> 10360 ： loss 0.056
epcho --> 10365 ： loss 0.056
epcho --> 10370 ： loss 0.056
epcho --> 10375 ： loss 0.056
epcho --> 10380 ： loss 0.056
epcho --> 10385 ： loss 0.056
epcho --> 10390 ： loss 0.056
epcho --> 10395 ： loss 0.056
epcho --> 10400 ： loss 0.056
epcho --> 10405 ： loss 0.056
epcho --> 10410 ： loss 0.056
epcho --> 10415 ： loss 0.056
epcho --> 10420 ： loss 0.056
epcho --> 10425 ： loss 0.056
epcho --> 10430 ： loss 0.056
epcho --> 10435 ： loss 0.056
epcho --> 10440 ： loss 0.056
epcho --> 10445 ： loss 0.056
epcho --> 10450 ： loss 0.057
epcho --> 1045

In [6]:
print(decode)
print(target)

tensor([[-0.2571, -0.2636, -0.2650, -0.2605, -0.3169, -0.2908, -0.6318, -0.4533,
         -0.4420, -0.1194],
        [-1.2194, -1.1845, -1.1930, -1.2207,  0.5459,  0.1038,  0.7714, -0.5541,
         -0.8477,  0.9763],
        [-1.0904, -1.0912, -1.0958, -1.0954, -0.3427, -0.5237, -0.5420, -0.4528,
         -0.4383, -0.1299],
        [-0.7954, -0.8040, -0.8044, -0.7985, -0.4298, -0.4952, -1.3418, -1.4378,
         -1.5266,  0.0809],
        [-0.9357, -0.9489, -0.9545, -0.9460, -0.5109, -0.5838, -0.9815, -0.2808,
         -0.0734, -0.8035],
        [ 0.2880,  0.3267,  0.2969,  0.2479,  2.6068,  2.5594, -0.1802, -0.1576,
          0.1135, -1.0116],
        [-0.1651, -0.1829, -0.1901, -0.1802, -0.3653, -0.2724, -0.8293,  0.1698,
          0.5403, -1.3222],
        [ 1.3665,  1.3738,  1.3652,  1.3559,  0.1619,  0.3871, -0.6694, -1.5177,
         -1.6189,  0.1139]], grad_fn=<ThAddmmBackward>)
tensor([[-0.2587, -0.3029, -0.2897, -0.2864, -0.6152, -0.5640, -0.8206, -0.3655,
         -0.3894,  